In [48]:
import gensim
from gensim.utils import simple_preprocess
import nltk.corpus
import nltk
import pandas as pd
import gensim.corpora
from langdetect import detect
from googletrans import Translator
from nltk.corpus import stopwords
import numpy
import time

In [49]:
data = pd.read_csv("final_data_cleaned.csv")
scam = data[data['y']==1]
scam.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2843 entries, 5611 to 8453
Data columns (total 12 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   username                        2843 non-null   object 
 1   age                             2841 non-null   float64
 2   occupation                      2839 non-null   object 
 3   status                          2843 non-null   object 
 4   gender                          2843 non-null   object 
 5   filtered sentence               2843 non-null   object 
 6   Language                        2843 non-null   object 
 7   y                               2843 non-null   int64  
 8   translated_occupation           2843 non-null   object 
 9   status_lem                      2843 non-null   object 
 10  translated_occupation_lem       2843 non-null   object 
 11  translated_occupation_category  2843 non-null   object 
dtypes: float64(1), int64(1), object

In [50]:
real = data[data['y']==0]
real.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5611 entries, 0 to 5610
Data columns (total 12 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   username                        5611 non-null   object 
 1   age                             5611 non-null   float64
 2   occupation                      5611 non-null   object 
 3   status                          5611 non-null   object 
 4   gender                          5611 non-null   object 
 5   filtered sentence               5611 non-null   object 
 6   Language                        5611 non-null   object 
 7   y                               5611 non-null   int64  
 8   translated_occupation           5611 non-null   object 
 9   status_lem                      5611 non-null   object 
 10  translated_occupation_lem       5611 non-null   object 
 11  translated_occupation_category  5611 non-null   object 
dtypes: float64(1), int64(1), object(10

In [51]:
data['y'].value_counts(normalize=True)*100

0    66.370949
1    33.629051
Name: y, dtype: float64

In [52]:
data['filtered sentence']

0                               I full fire friskier hell
1       ME GUSTA CONOCER CHICAS BONDADOSAS CARIÑOSAS Y...
2                        laid back earth good sense humor
3                Retired owner aerospace consuloting firm
4       Hello aaron Vermont I vermont 2 years Im reall...
                              ...                        
8449    Im earth caring honest going loves fun vacatio...
8450    I never know I write part lameSo Im ordinary n...
8451    I serious mature woman good valueslearned home...
8452    Im good woman looking good man One best friend...
8453    I would like meet man become life partner Some...
Name: filtered sentence, Length: 8454, dtype: object

In [53]:
def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

In [54]:
filtered_words = list(sent_to_words(scam['filtered sentence']))
filtered_words

[['cool',
  'simple',
  'easy',
  'going',
  'easy',
  'please',
  'treated',
  'resp',
  'ect',
  'cool',
  'simple',
  'easy',
  'going',
  'easy',
  'please',
  'treated',
  'resp',
  'ecti',
  'cool',
  'simple',
  'easy',
  'going',
  'easy',
  'please',
  'treated',
  'resp',
  'ect'],
 ['kind',
  'honest',
  'love',
  'caring',
  'man',
  'hobby',
  'music',
  'sport',
  'believe',
  'god'],
 ['as',
  'know',
  'myriads',
  'someone',
  'lots',
  'interested',
  'women',
  'choice',
  'choose',
  'onethe',
  'one',
  'heart',
  'goes',
  'forsomeone',
  'lean',
  'onto',
  'give',
  'love',
  'always',
  'wanted',
  'haveso',
  'tell',
  'yourselfwhat',
  'everything',
  'need',
  'tell',
  'yourselfand',
  'lets',
  'see',
  'goes',
  'herebut',
  'anywayheres',
  'little',
  'info',
  'like',
  'go',
  'dinner',
  'go',
  'dancinggo',
  'movies',
  'workout',
  'travel',
  'see',
  'new',
  'things',
  'enjoy',
  'simple',
  'things',
  'like',
  'ocean',
  'cuddling',
  'home

In [55]:
import gensim.corpora as corpora
# # Create Dictionary
id2word = corpora.Dictionary(filtered_words)
# Create Corpus
texts = filtered_words
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
# View
print(corpus[:1][0][:30])


[(0, 3), (1, 6), (2, 2), (3, 1), (4, 3), (5, 3), (6, 3), (7, 3), (8, 3)]


In [56]:
from pprint import pprint
# number of topics
num_topics = 5
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.021*"love" + 0.020*"life" + 0.012*"man" + 0.012*"like" + 0.009*"want" + '
  '0.008*"someone" + 0.008*"new" + 0.007*"good" + 0.007*"woman" + '
  '0.007*"person"'),
 (1,
  '0.023*"love" + 0.017*"im" + 0.016*"good" + 0.015*"man" + 0.013*"life" + '
  '0.012*"looking" + 0.012*"like" + 0.010*"person" + 0.009*"someone" + '
  '0.008*"honest"'),
 (2,
  '0.026*"love" + 0.016*"man" + 0.013*"woman" + 0.011*"im" + 0.011*"like" + '
  '0.011*"person" + 0.011*"good" + 0.010*"life" + 0.010*"looking" + '
  '0.008*"caring"'),
 (3,
  '0.020*"like" + 0.015*"love" + 0.015*"man" + 0.014*"good" + 0.013*"woman" + '
  '0.012*"life" + 0.010*"honest" + 0.010*"person" + 0.010*"looking" + '
  '0.009*"im"'),
 (4,
  '0.023*"love" + 0.020*"life" + 0.020*"im" + 0.017*"like" + 0.010*"woman" + '
  '0.009*"looking" + 0.009*"someone" + 0.008*"enjoy" + 0.007*"people" + '
  '0.007*"want"')]


In [60]:
filtered_words = list(sent_to_words(real['filtered sentence']))
filtered_words

[['full', 'fire', 'friskier', 'hell'],
 ['me',
  'gusta',
  'conocer',
  'chicas',
  'bondadosas',
  'carinosas',
  'muy',
  'fuertes',
  'en',
  'la',
  'moral'],
 ['laid', 'back', 'earth', 'good', 'sense', 'humor'],
 ['retired', 'owner', 'aerospace', 'consuloting', 'firm'],
 ['hello',
  'aaron',
  'vermont',
  'vermont',
  'years',
  'im',
  'really',
  'fun',
  'person',
  'like',
  'camping',
  'bone',
  'fire',
  'etc'],
 ['problem', 'free', 'flexibleseek', 'love', 'happiness'],
 ['easy', 'going', 'highly', 'educated', 'life', 'beautiful', 'travels'],
 ['name',
  'abdulrahman',
  'person',
  'ethics',
  'religion',
  'work',
  'field',
  'journalism',
  'live',
  'cairo'],
 ['serious',
  'honest',
  'trustworthy',
  'like',
  'share',
  'love',
  'fan',
  'culture',
  'new',
  'different',
  'person'],
 ['my',
  'name',
  'david',
  'im',
  'years',
  'old',
  'divorced',
  'retired',
  'grownup',
  'children',
  'son',
  'daughters'],
 ['loving',
  'friendly',
  'person',
  'alwa

In [61]:
import gensim.corpora as corpora
# # Create Dictionary
id2word = corpora.Dictionary(filtered_words)
# Create Corpus
texts = filtered_words
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
# View
print(corpus[:1][0][:30])

[(0, 1), (1, 1), (2, 1), (3, 1)]


In [62]:
from pprint import pprint
# number of topics
num_topics = 5
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.074*"straight" + 0.066*"nonsmoker" + 0.053*"never" + 0.044*"relationship" '
  '+ 0.041*"serious" + 0.041*"friendship" + 0.039*"white" + 0.038*"christian" '
  '+ 0.032*"other" + 0.027*"fun"'),
 (1,
  '0.091*"straight" + 0.086*"nonsmoker" + 0.062*"serious" + '
  '0.061*"relationship" + 0.056*"hispanic" + 0.048*"never" + 0.046*"drinker" + '
  '0.045*"christian" + 0.033*"friendship" + 0.028*"other"'),
 (2,
  '0.084*"straight" + 0.082*"drinker" + 0.075*"social" + 0.060*"serious" + '
  '0.060*"relationship" + 0.059*"nonsmoker" + 0.054*"hispanic" + '
  '0.053*"romance" + 0.051*"christian" + 0.047*"friendship"'),
 (3,
  '0.032*"straight" + 0.027*"relationship" + 0.026*"smoker" + 0.025*"drinker" '
  '+ 0.024*"serious" + 0.023*"light" + 0.023*"other" + 0.019*"occasional" + '
  '0.015*"friendship" + 0.015*"fun"'),
 (4,
  '0.024*"social" + 0.023*"relationship" + 0.021*"serious" + 0.018*"straight" '
  '+ 0.015*"drinker" + 0.013*"white" + 0.012*"smoker" + 0.011*"marriage" + '
  '0.009*"non